# evaluation
ragas ref:


https://docs.ragas.io/en/stable/index.html

In [1]:
from dotenv import load_dotenv
import os
import sys
import logging

from llama_index.core import StorageContext, load_index_from_storage

from llama_index.core.retrievers import VectorIndexRetriever

from llama_index.core import Settings

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

from llama_index.core.data_structs import Node
from llama_index.core.schema import NodeWithScore
from llama_index.core import get_response_synthesizer

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
#from ragas.metrics.critique import harmfulness


from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from datasets import Dataset
from ragas.integrations.llama_index import evaluate




import openai

import nest_asyncio
nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

import pandas as pd
from IPython.display import display, HTML


load_dotenv()  # Load environment variables from .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

OPENAI_API_KEY = openai.api_key

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# import logging
# logging.getLogger("httpx").setLevel(logging.WARNING)

%load_ext autoreload
%autoreload 2




/Users/juan/Documents/proyectos_propios/jobs/teamstation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Rebuild the storage context
storage_context = StorageContext.from_defaults(persist_dir="./persist")

# Load the index from storage
index = load_index_from_storage(storage_context)



def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = RankGPTRerank(
            llm=OpenAI(
               model="gpt-3.5-turbo",
               temperature=0.0,
               api_key=OPENAI_API_KEY,
            ),
            top_n=reranker_top_n,
            verbose=True,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes



query = "What are potential CONSECUENCES of keytruda?"
retrieved_nodes = get_retrieved_nodes(
    query,
    vector_top_k=30,
    reranker_top_n=5,
    with_reranker=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")

response = response_synthesizer.synthesize(
    query, nodes=retrieved_nodes
)

response

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
After Reranking, new rank list for nodes: [0, 6, 3, 1, 2, 7, 8, 25, 13, 23, 27, 24, 9, 14, 15, 16, 20, 18, 19, 21, 10, 4, 17, 26, 28, 29, 5, 11, 12, 22]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='Potential consequences of Keytruda include immune-related adverse effects such as colitis, hepatitis, pneumonitis, and exacerbation of pre-existing autoimmune diseases. It is important for patients to monitor for symptoms like cough, shortness of breath, chest pain, and changes in vision while on Keytruda treatment to ensure optimal care and safety.', source_nodes=[NodeWithScore(node=TextNode(id_='1c86f3e8-53f4-417a-9c1b-81fb42255e97', embedding=None, metadata={'document_title': 'Keytruda and Immune-Related Adverse Effects: A Comprehensive Overview'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='49de588d-beb9-4e7e-84dc-06ffb2e63a2d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='c73be1b4e2cb5e6157fd80000f0fe59f6ca8631fa4924c5ed0be7dd3180c3218')}, text='Can Keytruda cause immune-related adverse effects? Yes, Keytruda can cause immune-related adverse effects such as co

In [3]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

class RAGQueryEngine(CustomQueryEngine):
    """RAG Query Engine."""

    with_reranker: False
    response_synthesizer: BaseSynthesizer

    def custom_query(self, query_str: str):
        nodes = get_retrieved_nodes(
                        query_str,
                        vector_top_k=30,
                        reranker_top_n=5,
                        with_reranker=self.with_reranker,
                    )
        response_obj = self.response_synthesizer.synthesize(query_str, nodes)
        return response_obj
    


query_engine_without_reranker = RAGQueryEngine(
    response_synthesizer=get_response_synthesizer(response_mode="compact"),
    with_reranker = False
)

query_engine_with_reranker = RAGQueryEngine(
    response_synthesizer=get_response_synthesizer(response_mode="compact"),
    with_reranker = True
)



/Users/juan/Documents/proyectos_propios/jobs/teamstation/.venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:547: UserWarning: False is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [4]:
response = query_engine_without_reranker.query("What are potential CONSECUENCES of keytruda?")
response


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='Potential consequences of Keytruda include immune-related adverse effects such as colitis, hepatitis, and pneumonitis, as well as changes in blood pressure, interactions with steroids and immunosuppressants, and the need for close monitoring for specific symptoms like cough, chest pain, and changes in vision. Additionally, Keytruda may lead to fatigue, skin rash, nausea, and other common side effects.', source_nodes=[NodeWithScore(node=TextNode(id_='1c86f3e8-53f4-417a-9c1b-81fb42255e97', embedding=None, metadata={'document_title': 'Keytruda and Immune-Related Adverse Effects: A Comprehensive Overview'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='49de588d-beb9-4e7e-84dc-06ffb2e63a2d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='c73be1b4e2cb5e6157fd80000f0fe59f6ca8631fa4924c5ed0be7dd3180c3218')}, text='Can Keytruda cause immune-related adverse effects? Yes, Keytrud

In [5]:
response = query_engine_with_reranker.query("What are potential CONSECUENCES of keytruda?")
response


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
After Reranking, new rank list for nodes: [0, 6, 1, 3, 8, 25, 27, 23, 24, 9, 14, 13, 20, 18, 19, 21, 4, 10, 5, 7, 26, 17, 16, 15, 2, 11, 12, 28, 29, 22]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='Potential consequences of Keytruda include immune-related adverse effects such as colitis, hepatitis, pneumonitis, and the exacerbation of pre-existing autoimmune diseases. Regular monitoring of liver function, lung function, and immune markers is recommended to manage these potential side effects effectively.', source_nodes=[NodeWithScore(node=TextNode(id_='1c86f3e8-53f4-417a-9c1b-81fb42255e97', embedding=None, metadata={'document_title': 'Keytruda and Immune-Related Adverse Effects: A Comprehensive Overview'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='49de588d-beb9-4e7e-84dc-06ffb2e63a2d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='c73be1b4e2cb5e6157fd80000f0fe59f6ca8631fa4924c5ed0be7dd3180c3218')}, text='Can Keytruda cause immune-related adverse effects? Yes, Keytruda can cause immune-related adverse effects such as colitis, hepatitis, and pneumonitis.', mim

# Evaluation

In [6]:
df = pd.read_parquet("../data/processed_data/df_eval.parquet")


metrics = [
    faithfulness,
    answer_relevancy,
    #context_precision,
    #context_recall,
    #harmfulness,
]

evaluator_llm = OpenAI(model="gpt-3.5-turbo")


ds = df[["augmented_questions"]].rename(columns = {"augmented_questions":"question"})

ds_dict = ds.to_dict()
ds_dict["question"] = [v for k, v in ds_dict["question"].items()] 

dataset = Dataset.from_dict(ds_dict)
dataset

Dataset({
    features: ['question'],
    num_rows: 220
})

In [12]:
result = evaluate(
    query_engine=query_engine_without_reranker,
    metrics=metrics,
    dataset=dataset,
    llm=evaluator_llm,
    embeddings=OpenAIEmbedding(),
)

df_result_without_reranker = result.to_pandas()
df_result_without_reranker.to_parquet("../data/processed_data/df_result_ragas_without_reranking_.parquet")

df_result_without_reranker

In [11]:


result = evaluate(
    query_engine=query_engine_with_reranker,
    metrics=metrics,
    dataset=dataset,
    llm=evaluator_llm,
    embeddings=OpenAIEmbedding(),
)

df_result_with_reranker = result.to_pandas()
df_result_with_reranker.to_parquet("../data/processed_data/df_result_ragas_with_reranking_.parquet")

df_result_with_reranker

In [9]:
# without reranking
df_result_without_reranker.describe()

,faithfulness,answer_relevancy
count,218.000000,220.000000
mean,0.827316,0.859463
std,0.285554,0.301945
min,0.000000,0.000000
25%,0.600000,0.949778
50%,1.000000,0.964436
75%,1.000000,0.975003
max,1.000000,1.000000


In [10]:
# with reranking
df_result_with_reranker.describe()

,faithfulness,answer_relevancy
count,219.000000,220.000000
mean,0.789695,0.891236
std,0.321104,0.259486
min,0.000000,0.000000
25%,0.550000,0.949394
50%,1.000000,0.967308
75%,1.000000,0.979370
max,1.000000,1.000000
